In [1]:
import os
os.environ["CUDA_DEVICE_ORDER"] = "PCI_BUS_ID"
os.environ["CUDA_VISIBLE_DEVICES"] = "1"   #(xxxx is your specific GPU ID)

In [2]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications.resnet50 import preprocess_input
import numpy as np
import tensorflow as tf
from tensorflow.keras.applications import ResNet50
from tensorflow.keras.layers import Activation, Dense, Flatten, GlobalAveragePooling2D
from MyEarlyStopping import MyEarlyStopping
import matplotlib.pyplot as plt
from sklearn.metrics import confusion_matrix
import sklearn.metrics as metrics
from sklearn.preprocessing import LabelEncoder  
import pandas as pd

In [3]:
All_data = ImageDataGenerator(preprocessing_function=preprocess_input)
All_data_generator = All_data.flow_from_directory('Mel_Audio_folder_digits',
                                                batch_size=1,
                                                target_size=(224,224),
                                                class_mode='categorical',
                                               shuffle=False)

Found 30000 images belonging to 10 classes.


In [4]:
image_names = All_data_generator.filenames
image_no = [i.split("/")[1].split("_")[2].split(".")[0] for i in image_names]
image_no = np.array(list(map(int, image_no)))
ALL_participant_class = [i.split("/")[1].split("_")[1] for i in image_names]
ALL_participant_class = np.array(list(map(int, ALL_participant_class)))
command_class = All_data_generator.classes
All_participant_class = tf.keras.utils.to_categorical(ALL_participant_class-1, num_classes=60)
All_command_class = tf.keras.utils.to_categorical(command_class, num_classes=10)
All_command_uniform = All_command_class*0+1/10

In [5]:
All_Inputs = [next(All_data_generator)[0][0] for _ in range(len(All_data_generator))]
All_Inputs = np.array(All_Inputs)

In [6]:
# image_no<=20 means every word repeat 20 times 
s_number = 5 # ALL_participant_class  the subject index

select_indexs_train = (image_no<=19)&(ALL_participant_class<=s_number)
Train_Inputs = All_Inputs[select_indexs_train]
Train_participant_class = All_participant_class[select_indexs_train][:,0:s_number]
Train_participant_uniform = Train_participant_class*0+1/s_number
Train_command_class = All_command_class[select_indexs_train]
Train_command_uniform = Train_command_class*0+1/10

In [7]:
select_indexs_val = (image_no>29)&(image_no<=39)&(ALL_participant_class<=s_number)
Val_Inputs = All_Inputs[select_indexs_val]
Val_participant_class = All_participant_class[select_indexs_val][:,0:s_number]
Val_participant_uniform = Val_participant_class*0+1/s_number
Val_command_class = All_command_class[select_indexs_val]
Val_command_uniform = Val_command_class*0+1/10

In [8]:
select_indexs_test = (image_no>39)&(image_no<=49)&(ALL_participant_class<=s_number)
Test_Inputs = All_Inputs[select_indexs_test]
Test_participant_class = All_participant_class[select_indexs_test][:,0:s_number]
Test_participant_uniform = Test_participant_class*0+1/s_number
Test_command_class = All_command_class[select_indexs_test]
Test_command_uniform = Test_command_class*0+1/10

In [9]:
actv_fun_1_1 = "relu" 
actv_fun_1_2 = "sigmoid"
shape_1_1 = 128
shape_1_2 = 256
actv_fun_2_1 = "sigmoid" 
actv_fun_2_2 = "sigmoid"
shape_2_1 = 512
shape_2_2 = 512


In [10]:
high_acc = 0
for run in range(0,10):
    
    # feature extraction layers
    resnet_model = ResNet50(input_shape=(224, 224,3), include_top=False, weights="imagenet")
    for layer in resnet_model.layers:
        layer.trainable = False

    d = resnet_model.output.shape[-1] # dimension of last layer

    ###################### model 1 ###################### 
    layer_1_0 = tf.keras.layers.Dense(d,name="weight_1")(resnet_model.output) #times weight before flatten
    layer_1_1 = tf.keras.layers.Flatten(name='flatten_1')(layer_1_0)

    Dense_1_1 = tf.keras.layers.Dense(shape_1_1, activation=actv_fun_1_1,name='fc1_1')
    layer_1_2 = Dense_1_1(layer_1_1)
    Dense_1_2 = tf.keras.layers.Dense(shape_1_2, activation=actv_fun_1_2,name='fc1_2')
    layer_1_3 = Dense_1_2(layer_1_2)

    Dense_1_3 = tf.keras.layers.Dense(s_number, activation='softmax' ,name='participant_output')
    out_layer_1 = Dense_1_3(layer_1_3)

    ###################### model 2 ###################### 
    layer_2_0 = tf.keras.layers.Dense(d,name="weight_2")(resnet_model.output) #times weight before flatten
    layer_2_1 = tf.keras.layers.Flatten(name='flatten_2')(layer_2_0)

    Dense_2_1 = tf.keras.layers.Dense(shape_2_1, activation=actv_fun_2_1,name='fc2_1')
    layer_2_2  = Dense_2_1(layer_2_1)
    Dense_2_2 = tf.keras.layers.Dense(shape_2_2, activation=actv_fun_2_2,name='fc2_2')
    layer_2_3  = Dense_2_2(layer_2_2)

    Dense_2_3 = tf.keras.layers.Dense(10, activation='softmax',name='command_output')
    out_layer_2 = Dense_2_3(layer_2_3)

    ###################### model 1' ###################### 
    layer_1_2_  = Dense_2_1(layer_1_1)
    layer_1_3_  = Dense_2_2(layer_1_2_)
    out_layer_1_ = Dense_2_3(layer_1_3_)

    ###################### model 1' ###################### 
    layer_2_2_  = Dense_1_1(layer_2_1)
    layer_2_3_  = Dense_1_2(layer_2_2_)
    out_layer_2_ = Dense_1_3(layer_2_3_)

    resnet_model = tf.keras.Model(resnet_model.input, [out_layer_1,out_layer_2,out_layer_1_,out_layer_2_])

    # resnet_model.summary() 

    w_1, w_2, w_1_, w_2_ = 1,1,1,1
    ##################### training ############################
    opt = tf.keras.optimizers.Adam(learning_rate=0.0001)
    callbacks = [MyEarlyStopping(monitor1 = 'val_' + resnet_model.layers[-1].name+'_accuracy',
                                  monitor2 = 'val_' + resnet_model.layers[-2].name+'_accuracy',
                                  patience=5,restore_best_weights=True)]
    resnet_model.compile(optimizer=opt, loss=["categorical_crossentropy","categorical_crossentropy","mse","mse"],
                         loss_weights=[w_1, w_2, w_1_, w_2_], metrics=['accuracy'])
    history1 = resnet_model.fit(Train_Inputs, 
                               {resnet_model.layers[-2].name:Train_participant_class, 
                                resnet_model.layers[-1].name:Train_command_class,
                                resnet_model.layers[-1].name+"_1":Train_command_uniform, 
                                resnet_model.layers[-2].name+"_1":Train_participant_uniform}, 
                                validation_data=(Val_Inputs,
                                                 {resnet_model.layers[-2].name:Val_participant_class,
                                                  resnet_model.layers[-1].name:Val_command_class,
                                                  resnet_model.layers[-1].name+"_1":Val_command_uniform,
                                                  resnet_model.layers[-2].name+"_1":Val_participant_uniform}), 
                               callbacks=callbacks,
                               batch_size=64,
                               epochs=10)

    for layer in resnet_model.layers[0:175]:
        layer.trainable = True
    history2 = resnet_model.fit(Train_Inputs, 
                               {resnet_model.layers[-2].name:Train_participant_class, 
                                resnet_model.layers[-1].name:Train_command_class,
                                resnet_model.layers[-1].name+"_1":Train_command_uniform, 
                                resnet_model.layers[-2].name+"_1":Train_participant_uniform}, 
                                validation_data=(Val_Inputs,
                                                 {resnet_model.layers[-2].name:Val_participant_class,
                                                  resnet_model.layers[-1].name:Val_command_class,
                                                  resnet_model.layers[-1].name+"_1":Val_command_uniform,
                                                  resnet_model.layers[-2].name+"_1":Val_participant_uniform}), 
                               callbacks=callbacks,
                               batch_size=64,
                               epochs=10)
   
    predictions = resnet_model.predict(Test_Inputs)[0]
    predicted_classes = np.argmax(predictions, axis=1)+1
    participant_class = [i.split("/")[1].split("_")[1] for i in All_data_generator.filenames]
    participant_class = np.array(participant_class)[select_indexs_test]
    acc_c = round(sum(x == y for x, y in zip(participant_class, predicted_classes)) / len(participant_class),4)
    
    predictions = resnet_model.predict(Test_Inputs)[1]
    predicted_classes = np.argmax(predictions, axis=1)
    true_classes = np.array([i.split("/")[0] for i in All_data_generator.filenames])
    true_classes = np.array(list(map(int, true_classes)))[select_indexs_test]
    acc_s = round(sum(x == y for x, y in zip(true_classes, predicted_classes)) / len(true_classes),4)
    overall_acc = acc_c + acc_s

    
    if high_acc < overall_acc:
        resnet_model.save('Digital_data_Initial_group_model_0531_free&unfree.h5')
        high_acc = overall_acc
        best_index = run
        
    del resnet_model
    run = run + 1

Epoch 1/10
16/16 [==============================] - 5s 304ms/step - loss: 3.4133 - participant_output_loss: 1.3807 - command_output_loss: 2.0311 - command_output_1_loss: 4.1223e-04 - participant_output_1_loss: 0.0011 - participant_output_accuracy: 0.5510 - command_output_accuracy: 0.5350 - command_output_1_accuracy: 0.1440 - participant_output_1_accuracy: 0.2250 - val_loss: 2.8154 - val_participant_output_loss: 1.0313 - val_command_output_loss: 1.7839 - val_command_output_1_loss: 5.2758e-05 - val_participant_output_1_loss: 1.5149e-04 - val_participant_output_accuracy: 0.8300 - val_command_output_accuracy: 0.9340 - val_command_output_1_accuracy: 0.0020 - val_participant_output_1_accuracy: 0.4540
Epoch 2/10
16/16 [==============================] - 3s 167ms/step - loss: 2.5472 - participant_output_loss: 0.8657 - command_output_loss: 1.6813 - command_output_1_loss: 6.0388e-05 - participant_output_1_loss: 1.1314e-04 - participant_output_accuracy: 0.8920 - command_output_accuracy: 0.9630 - c

Epoch 3/10
16/16 [==============================] - 2s 134ms/step - loss: 0.4379 - participant_output_loss: 0.1088 - command_output_loss: 0.3290 - command_output_1_loss: 2.9274e-05 - participant_output_1_loss: 2.2894e-05 - participant_output_accuracy: 1.0000 - command_output_accuracy: 1.0000 - command_output_1_accuracy: 0.0280 - participant_output_1_accuracy: 0.2080 - val_loss: 0.4872 - val_participant_output_loss: 0.1567 - val_command_output_loss: 0.3305 - val_command_output_1_loss: 1.3411e-05 - val_participant_output_1_loss: 3.5734e-05 - val_participant_output_accuracy: 0.9920 - val_command_output_accuracy: 0.9960 - val_command_output_1_accuracy: 0.0000e+00 - val_participant_output_1_accuracy: 0.2900
Epoch 4/10
16/16 [==============================] - 2s 125ms/step - loss: 0.3864 - participant_output_loss: 0.1000 - command_output_loss: 0.2864 - command_output_1_loss: 2.8290e-05 - participant_output_1_loss: 2.0939e-05 - participant_output_accuracy: 1.0000 - command_output_accuracy: 1.

Epoch 9/10
16/16 [==============================] - 2s 154ms/step - loss: 1.4187 - participant_output_loss: 0.2588 - command_output_loss: 1.1598 - command_output_1_loss: 5.3725e-05 - participant_output_1_loss: 2.3957e-05 - participant_output_accuracy: 0.9990 - command_output_accuracy: 0.9990 - command_output_1_accuracy: 0.4250 - participant_output_1_accuracy: 0.1560 - val_loss: 1.4293 - val_participant_output_loss: 0.3001 - val_command_output_loss: 1.1291 - val_command_output_1_loss: 3.8559e-05 - val_participant_output_1_loss: 2.8539e-05 - val_participant_output_accuracy: 0.9860 - val_command_output_accuracy: 0.9960 - val_command_output_1_accuracy: 0.5160 - val_participant_output_1_accuracy: 0.1660
Epoch 10/10
16/16 [==============================] - 3s 212ms/step - loss: 1.2866 - participant_output_loss: 0.2180 - command_output_loss: 1.0686 - command_output_1_loss: 7.8619e-05 - participant_output_1_loss: 2.3000e-05 - participant_output_accuracy: 0.9990 - command_output_accuracy: 0.999

Epoch 4/10
16/16 [==============================] - 3s 178ms/step - loss: 2.4136 - participant_output_loss: 0.7047 - command_output_loss: 1.7088 - command_output_1_loss: 5.0722e-05 - participant_output_1_loss: 3.2903e-05 - participant_output_accuracy: 0.9730 - command_output_accuracy: 0.9890 - command_output_1_accuracy: 0.0010 - participant_output_1_accuracy: 0.2370 - val_loss: 2.3359 - val_participant_output_loss: 0.6809 - val_command_output_loss: 1.6550 - val_command_output_1_loss: 4.0835e-05 - val_participant_output_1_loss: 3.1999e-05 - val_participant_output_accuracy: 0.9500 - val_command_output_accuracy: 0.9880 - val_command_output_1_accuracy: 0.0020 - val_participant_output_1_accuracy: 0.0860
Epoch 5/10
16/16 [==============================] - 3s 195ms/step - loss: 2.1702 - participant_output_loss: 0.5754 - command_output_loss: 1.5947 - command_output_1_loss: 6.6689e-05 - participant_output_1_loss: 2.7149e-05 - participant_output_accuracy: 0.9890 - command_output_accuracy: 0.9940

Epoch 6/10
16/16 [==============================] - 3s 191ms/step - loss: 0.7291 - participant_output_loss: 0.1221 - command_output_loss: 0.6069 - command_output_1_loss: 5.9889e-05 - participant_output_1_loss: 1.2503e-05 - participant_output_accuracy: 0.9990 - command_output_accuracy: 1.0000 - command_output_1_accuracy: 0.0000e+00 - participant_output_1_accuracy: 0.2070 - val_loss: 0.7677 - val_participant_output_loss: 0.1690 - val_command_output_loss: 0.5986 - val_command_output_1_loss: 1.5255e-05 - val_participant_output_1_loss: 1.5596e-05 - val_participant_output_accuracy: 0.9940 - val_command_output_accuracy: 0.9980 - val_command_output_1_accuracy: 0.0040 - val_participant_output_1_accuracy: 0.2960
Epoch 1/10
16/16 [==============================] - 4s 252ms/step - loss: 3.6799 - participant_output_loss: 1.5091 - command_output_loss: 2.1684 - command_output_1_loss: 3.1363e-04 - participant_output_1_loss: 0.0021 - participant_output_accuracy: 0.4090 - command_output_accuracy: 0.4190

...

Epoch 5/10
16/16 [==============================] - 3s 160ms/step - loss: 0.6958 - participant_output_loss: 0.0442 - command_output_loss: 0.6516 - command_output_1_loss: 1.8898e-05 - participant_output_1_loss: 1.0914e-05 - participant_output_accuracy: 1.0000 - command_output_accuracy: 1.0000 - command_output_1_accuracy: 0.1600 - participant_output_1_accuracy: 0.2180 - val_loss: 0.7257 - val_participant_output_loss: 0.0873 - val_command_output_loss: 0.6383 - val_command_output_1_loss: 3.8570e-05 - val_participant_output_1_loss: 1.4624e-05 - val_participant_output_accuracy: 0.9920 - val_command_output_accuracy: 0.9980 - val_command_output_1_accuracy: 0.0000e+00 - val_participant_output_1_accuracy: 0.1640
Epoch 6/10
16/16 [==============================] - 2s 156ms/step - loss: 0.6282 - participant_output_loss: 0.0406 - command_output_loss: 0.5876 - command_output_1_loss: 3.5173e-05 - participant_output_1_loss: 9.9807e-06 - participant_output_accuracy: 1.0000 - command_output_accuracy: 1.

16/16 [==============================] - 2s 139ms/step - loss: 2.1275 - participant_output_loss: 0.3592 - command_output_loss: 1.7682 - command_output_1_loss: 6.2560e-05 - participant_output_1_loss: 2.5059e-05 - participant_output_accuracy: 0.9960 - command_output_accuracy: 0.9920 - command_output_1_accuracy: 0.0070 - participant_output_1_accuracy: 0.1810 - val_loss: 2.1270 - val_participant_output_loss: 0.3935 - val_command_output_loss: 1.7335 - val_command_output_1_loss: 4.0147e-05 - val_participant_output_1_loss: 2.6725e-05 - val_participant_output_accuracy: 0.9820 - val_command_output_accuracy: 0.9940 - val_command_output_1_accuracy: 0.1860 - val_participant_output_1_accuracy: 0.1720
Epoch 7/10
16/16 [==============================] - 2s 156ms/step - loss: 1.9854 - participant_output_loss: 0.2965 - command_output_loss: 1.6888 - command_output_1_loss: 6.3041e-05 - participant_output_1_loss: 2.1980e-05 - participant_output_accuracy: 0.9980 - command_output_accuracy: 0.9970 - command_

Epoch 2/10
16/16 [==============================] - 2s 144ms/step - loss: 2.8150 - participant_output_loss: 0.9712 - command_output_loss: 1.8436 - command_output_1_loss: 6.6427e-05 - participant_output_1_loss: 1.5331e-04 - participant_output_accuracy: 0.8620 - command_output_accuracy: 0.9170 - command_output_1_accuracy: 0.0000e+00 - participant_output_1_accuracy: 0.3360 - val_loss: 2.5879 - val_participant_output_loss: 0.8284 - val_command_output_loss: 1.7593 - val_command_output_1_loss: 2.8577e-05 - val_participant_output_1_loss: 9.7382e-05 - val_participant_output_accuracy: 0.8940 - val_command_output_accuracy: 0.9580 - val_command_output_1_accuracy: 0.0140 - val_participant_output_1_accuracy: 0.0020
Epoch 3/10
16/16 [==============================] - 2s 150ms/step - loss: 2.3567 - participant_output_loss: 0.6728 - command_output_loss: 1.6838 - command_output_1_loss: 5.5312e-05 - participant_output_1_loss: 5.5240e-05 - participant_output_accuracy: 0.9590 - command_output_accuracy: 0.

Epoch 4/10
16/16 [==============================] - 2s 144ms/step - loss: 0.6280 - participant_output_loss: 0.0873 - command_output_loss: 0.5406 - command_output_1_loss: 2.2835e-05 - participant_output_1_loss: 1.1767e-05 - participant_output_accuracy: 1.0000 - command_output_accuracy: 1.0000 - command_output_1_accuracy: 0.1200 - participant_output_1_accuracy: 0.2230 - val_loss: 0.6706 - val_participant_output_loss: 0.1365 - val_command_output_loss: 0.5341 - val_command_output_1_loss: 2.2873e-05 - val_participant_output_1_loss: 1.4280e-05 - val_participant_output_accuracy: 0.9920 - val_command_output_accuracy: 0.9960 - val_command_output_1_accuracy: 0.3220 - val_participant_output_1_accuracy: 0.1440
Epoch 5/10
16/16 [==============================] - 2s 129ms/step - loss: 0.5652 - participant_output_loss: 0.0779 - command_output_loss: 0.4872 - command_output_1_loss: 2.8860e-05 - participant_output_1_loss: 1.0744e-05 - participant_output_accuracy: 1.0000 - command_output_accuracy: 1.0000

Epoch 10/10
16/16 [==============================] - 2s 126ms/step - loss: 1.2220 - participant_output_loss: 0.1513 - command_output_loss: 1.0706 - command_output_1_loss: 5.8860e-05 - participant_output_1_loss: 1.7052e-05 - participant_output_accuracy: 0.9990 - command_output_accuracy: 1.0000 - command_output_1_accuracy: 0.0010 - participant_output_1_accuracy: 0.2080 - val_loss: 1.2370 - val_participant_output_loss: 0.2022 - val_command_output_loss: 1.0348 - val_command_output_1_loss: 2.3293e-05 - val_participant_output_1_loss: 1.9141e-05 - val_participant_output_accuracy: 0.9920 - val_command_output_accuracy: 0.9960 - val_command_output_1_accuracy: 0.0200 - val_participant_output_1_accuracy: 0.2820
Epoch 1/10
16/16 [==============================] - 3s 205ms/step - loss: 1.1069 - participant_output_loss: 0.1371 - command_output_loss: 0.9698 - command_output_1_loss: 5.2271e-05 - participant_output_1_loss: 1.6608e-05 - participant_output_accuracy: 0.9990 - command_output_accuracy: 1.000

In [15]:
resnet_model = tf.keras.models.load_model('Digital_data_Initial_group_model_0531_free&unfree.h5')

In [12]:
# participant prediction
predictions = resnet_model.predict(Test_Inputs)[0]
predicted_classes = np.argmax(predictions, axis=1)+1
participant_class = [i.split("/")[1].split("_")[1] for i in All_data_generator.filenames]
participant_class = np.array(participant_class)[select_indexs_test]

true_classes = np.array(list(map(int, participant_class)))

print('Confusion Matrix')
print(confusion_matrix(true_classes, predicted_classes))

print('\nConfusion Matrix Report')
report = metrics.classification_report(true_classes, predicted_classes )
print(report)

Confusion Matrix
[[ 99   0   0   0   1]
 [  0  99   0   1   0]
 [  0   0 100   0   0]
 [  1   0   1  98   0]
 [  0   0   0   0 100]]

Confusion Matrix Report
              precision    recall  f1-score   support

           1       0.99      0.99      0.99       100
           2       1.00      0.99      0.99       100
           3       0.99      1.00      1.00       100
           4       0.99      0.98      0.98       100
           5       0.99      1.00      1.00       100

    accuracy                           0.99       500
   macro avg       0.99      0.99      0.99       500
weighted avg       0.99      0.99      0.99       500



In [13]:
# command prediction
predictions = resnet_model.predict(Test_Inputs)[1]
predicted_classes = np.argmax(predictions, axis=1)
true_classes = np.array([i.split("/")[0] for i in All_data_generator.filenames])
true_classes = np.array(list(map(int, true_classes)))[select_indexs_test]

print('Confusion Matrix')
print(confusion_matrix(true_classes, predicted_classes))

print('\nConfusion Matrix Report')
report = metrics.classification_report(true_classes, predicted_classes, target_names=list(All_data_generator.class_indices.keys()))
print(report)

Confusion Matrix
[[50  0  0  0  0  0  0  0  0  0]
 [ 0 50  0  0  0  0  0  0  0  0]
 [ 0  0 50  0  0  0  0  0  0  0]
 [ 0  0  0 50  0  0  0  0  0  0]
 [ 0  0  0  0 50  0  0  0  0  0]
 [ 0  0  0  0  0 50  0  0  0  0]
 [ 0  0  0  0  0  0 50  0  0  0]
 [ 0  0  0  0  0  0  0 50  0  0]
 [ 0  0  0  0  0  0  0  0 50  0]
 [ 0  1  0  0  0  0  0  0  0 49]]

Confusion Matrix Report
              precision    recall  f1-score   support

           0       1.00      1.00      1.00        50
           1       0.98      1.00      0.99        50
           2       1.00      1.00      1.00        50
           3       1.00      1.00      1.00        50
           4       1.00      1.00      1.00        50
           5       1.00      1.00      1.00        50
           6       1.00      1.00      1.00        50
           7       1.00      1.00      1.00        50
           8       1.00      1.00      1.00        50
           9       1.00      0.98      0.99        50

    accuracy                   